In [ ]:
!pip install datasets
!pip install transformers==4.28.0
!pip install spacy
!python -m spacy download de_core_news_lg

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset, DatasetDict

import torch

from sklearn import metrics as sk_metrics

import numpy as np
import pandas as pd
import re
from ast import literal_eval
from collections import Counter

from IPython.display import display, Markdown
from tqdm import tqdm

import spacy
from spacy.matcher import PhraseMatcher
tqdm.pandas()

In [3]:
id2label = {0: 'O', 1: 'PROF'}
label2id = {'O': 0, 'B-PROF': 1, 'I-PROF': 1}

validation_df = pd.read_csv('../data/test.csv',
                           encoding='utf-8', sep='\t', index_col=0)
validation_df.rename_axis('id', axis='index', inplace=True)

validation_df['tokens'] = validation_df['tokens'].apply(lambda x: literal_eval(x))
validation_df['annotations'] = validation_df['annotations'].apply(lambda x: literal_eval(x))
validation_df['annotations'] = validation_df['annotations'].apply(lambda x: [label2id[l] for l in x])

print(len(validation_df))
validation_df.head()

163


,tokens,annotations
id,,
0,"[Für, uns, sind, die, Alten-, und, Krankenpfle...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, ..."
5,"[Die, Interessen, eines, Arztes, sind, nicht, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,"[Nicht, nur, für, die, betroffenen, Frauen, mü...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,"[Dass, ein, Krankenpfleger, oder, eine, Kranke...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10,"[Viele, Pflegerinnen, und, Pfleger, in, unsere...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


# Initialize BERT model

In [5]:
model_checkpoint = 'johannabi/german_tc_professions_debates'
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="simple")

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

# Initialize PhraseMatchers
You have to download the dictionary first: https://download-portal.arbeitsagentur.de/files/ (File name: DKZ_Suchworte_Systematik_und_Berufe_gueltig.xml)

In [ ]:
nlp = spacy.load('de_core_news_lg', disable=['ner'])

dkz_path = '../data/DKZ_Suchworte_Systematik_und_Berufe_gueltig.xml'
dkz_df = pd.read_xml(dkz_path)

all_berufe_list = list(dkz_df['name'].unique())
matcher_all = PhraseMatcher(nlp.vocab, attr='LEMMA')
occ_nlp_all = list(nlp.pipe(all_berufe_list))

for occ, occ_proc in zip(all_berufe_list, occ_nlp_all):
    matcher_all.add(occ, [occ_proc])


dkz_mw_df = dkz_df[dkz_df.suchwortGruppe.isin(['m', 'w'])].copy()
berufe_list = list(dkz_mw_df['name'].unique())
matcher_mw = PhraseMatcher(nlp.vocab, attr='LEMMA')
occ_nlp_mw = list(nlp.pipe(berufe_list))

for occ, occ_proc in zip(berufe_list, occ_nlp_mw):
    matcher_mw.add(occ, [occ_proc])

matcher_token_mw = PhraseMatcher(nlp.vocab, attr='ORTH')
for occ, occ_proc in zip(berufe_list, occ_nlp_mw):
    matcher_token_mw.add(occ, [occ_proc])

In [9]:
print(len(all_berufe_list))
print(len(berufe_list))

183205
107020


# Use BERT model with different aggregation strategies

In [56]:
def predict_to_list(tokens, aggregation_strategy):
    string = ' '.join(tokens)    
    predictions = token_classifier(string, aggregation_strategy=aggregation_strategy)
    if len(predictions) == 0:
        return [0 for t in tokens]
    # idenfity start and end char of tokens
    start_end_list = list()
    pointer = 0
    for t in tokens:
        start_end_list.append((t, pointer, pointer+len(t)))
        pointer+=len(t)+1
    se_df = pd.DataFrame(start_end_list, columns=['token', 'start', 'end'])
    
    # align predictions with tokens
    annotations = [0] * len(tokens)
    for pred in predictions:
        #print(pred)
        try:
            #greedy annotation
            start_token = max(se_df[se_df.start <= pred['start']].index) # nearst possible under boundary
            end_token = min(se_df[se_df.end >= pred['end']].index) # nearst possible upper boundary

            for idx in range(start_token, end_token+1):
                annotations[idx] = 1
        except IndexError:
            print(pred, '\n', tokens)
    return annotations

validation_df['pred_bert_simple'] = validation_df['tokens'].progress_apply(lambda x: predict_to_list(x, aggregation_strategy='simple'))
validation_df['pred_bert_first'] = validation_df['tokens'].progress_apply(lambda x: predict_to_list(x, aggregation_strategy='first'))
validation_df['pred_bert_average'] = validation_df['tokens'].progress_apply(lambda x: predict_to_list(x, aggregation_strategy='average'))
validation_df['pred_bert_max'] = validation_df['tokens'].progress_apply(lambda x: predict_to_list(x, aggregation_strategy='max'))

100%|██████████| 163/163 [00:48<00:00,  3.33it/s]


# Use PhraseMatcher with different dictionaries and on OrthForm or Lemma level

In [30]:
def phrasematch_to_list(doc, matcher):
    matches = matcher(doc) 
    annotations = [0] * len(doc)
    
    for match_id, start, end in matches:
        for i in range(start, end):
            annotations[i] = 1
    return annotations

validation_df['spacydoc'] = validation_df['tokens'].progress_apply(lambda x: nlp(' '.join(x)))
validation_df['pred_matcher_mw'] = validation_df['spacydoc'].progress_apply(lambda x: phrasematch_to_list(x, matcher_mw))
validation_df['pred_matcher_mw_t'] = validation_df['spacydoc'].progress_apply(lambda x: phrasematch_to_list(x, matcher_token_mw))
validation_df['pred_matcher_all'] = validation_df['spacydoc'].progress_apply(lambda x: phrasematch_to_list(x, matcher_all))
validation_df.head()

100%|██████████| 163/163 [00:00<00:00, 48872.08it/s]


,tokens,annotations,pred_bert_simple,pred_bert_first,pred_bert_average,spacydoc,pred_matcher_mw,pred_matcher_mw_t,pred_matcher_all
id,,,,,,,,,
0,"[Für, uns, sind, die, Alten-, und, Krankenpfle...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","(Für, uns, sind, die, Alten-, und, Krankenpfle...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, ..."
5,"[Die, Interessen, eines, Arztes, sind, nicht, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","(Die, Interessen, eines, Arztes, sind, nicht, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
7,"[Nicht, nur, für, die, betroffenen, Frauen, mü...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(Nicht, nur, für, die, betroffenen, Frauen, mü...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
9,"[Dass, ein, Krankenpfleger, oder, eine, Kranke...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(Dass, ein, Krankenpfleger, oder, eine, Kranke...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
10,"[Viele, Pflegerinnen, und, Pfleger, in, unsere...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(Viele, Pflegerinnen, und, Pfleger, in, unsere...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ..."


# Evaluate all Experiments at Token Level

In [57]:
exploded = validation_df.explode(['tokens', 'annotations', 
                                  'pred_matcher_mw', 'pred_matcher_mw_t','pred_matcher_all',
                                  'pred_bert_simple', 'pred_bert_first', 'pred_bert_average', 'pred_bert_max'])

In [58]:
experiments = ['pred_matcher_mw', 'pred_matcher_mw_t','pred_matcher_all',
               'pred_bert_simple', 'pred_bert_first', 'pred_bert_average', 'pred_bert_max']

series_list = list()

for e in experiments:
    result = sk_metrics.classification_report(list(exploded.annotations), 
                              list(exploded[e]), labels=[0,1], output_dict=True)['1']
    #result['name'] = e
    s = pd.Series(result, name=e)
    series_list.append(s)
result_df = pd.DataFrame(series_list)
result_df.sort_values('f1-score')

,precision,recall,f1-score,support
pred_matcher_all,0.260644,0.784375,0.391270,320.0
pred_matcher_mw_t,0.925000,0.462500,0.616667,320.0
pred_matcher_mw,0.935361,0.768750,0.843911,320.0
pred_bert_simple,0.926174,0.862500,0.893204,320.0
pred_bert_first,0.932432,0.862500,0.896104,320.0
pred_bert_average,0.932432,0.862500,0.896104,320.0
pred_bert_max,0.935593,0.862500,0.897561,320.0


# Qualitative Error Analysis

In [59]:
validation_df[validation_df.pred_bert_max != validation_df.pred_bert_average]

,tokens,annotations,pred_bert_simple,pred_bert_first,pred_bert_average,spacydoc,pred_matcher_mw,pred_matcher_mw_t,pred_matcher_all,pred_bert_max
id,,,,,,,,,,
73,"[Wir, beraten, heute, in, erster, Lesung, , –...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(Wir, beraten, heute, in, erster, Lesung, , ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [62]:
for i, row in validation_df.iterrows():
    #if row.annotations == row.pred_bert == row.pred_matcher:
    #    continue
    
    tokens_bert = list(row.tokens)
    predicted_bert = list(row.pred_bert_max)
    tokens_matcher = list(row.tokens)
    predicted_matcher = list(row.pred_matcher_mw)
    true = list(row.annotations)
    relevant = False
    if 'Verkäufer' in tokens_bert:
        relevant = True
    for t_i, t in enumerate(true):
        tp_string = "<font color='green'> {} </font>"
        fp_string = "<font color='red'> {} </font>"
        fn_string = "<font color='orange'> {} </font>"

        p_b = predicted_bert[t_i]
        p_m = predicted_matcher[t_i]
        if t == 1:
            if p_b == 1: #tp
                tokens_bert[t_i] = tp_string.format(tokens_bert[t_i])
            else: #fn
                tokens_bert[t_i] = fn_string.format(tokens_bert[t_i])

            if p_m == 1: #tp
                tokens_matcher[t_i] = tp_string.format(tokens_matcher[t_i])
            else: #fn
                tokens_matcher[t_i] = fn_string.format(tokens_matcher[t_i]) 
        else:
            if p_b == 1: #fp
                tokens_bert[t_i] = fp_string.format(tokens_bert[t_i])
            if p_m == 1: #fp
                tokens_matcher[t_i] = fp_string.format(tokens_matcher[t_i]) 

            
    if relevant:
        print(i)
        display(Markdown('B:' + ' '.join(tokens_bert)))

        display(Markdown('M:' + ' '.join(tokens_matcher)))
        print('###\n\n')

208


B:Änderungsbedarf sieht er darin , dass im Rahmen der Aktualisierungspflicht dem Verbraucher ein Anspruch auf Updates nicht nur gegenüber dem Verkäufer , sondern zusätzlich auch gegenüber dem Hersteller zustehen soll und dass die zweijährige Verjährung von Ansprüchen im Zusammenhang mit Mängeln an digitalen Elementen nicht erst mit Ablauf des Aktualisierungszeitraumes zu laufen beginnen soll .  

M:Änderungsbedarf sieht er darin , dass im Rahmen der Aktualisierungspflicht dem Verbraucher ein Anspruch auf Updates nicht nur gegenüber dem <font color='red'> Verkäufer </font> , sondern zusätzlich auch gegenüber dem <font color='red'> Hersteller </font> zustehen soll und dass die zweijährige Verjährung von Ansprüchen im Zusammenhang mit Mängeln an digitalen Elementen nicht erst mit Ablauf des Aktualisierungszeitraumes zu laufen beginnen soll .  

###


281


B:Es ist mit den staatlichen und den Hoheitsaufgaben des <font color='orange'> Beamten </font> einfach nicht vereinbar , daß er sich nebenbei noch als <font color='green'> Verkäufer </font> betätigt .

...

M:Statt den Kaufpreis in bar entgegenzunehmen , hat der <font color='red'> Verkäufer </font> künftig die Möglichkeit , Wohnrecht sowie eine Geld- und Naturalrente -in Form etwa des üblichen Altenteils - auf Lebenszeit mit dem Übernehmer zu vereinbaren .

###


762


B:Ihre <font color='orange'> Verkäufer </font> sollen sich gefälligst darum kümmern , daß an demokratische Rechtsstaaten geliefert wird .

M:Ihre <font color='green'> Verkäufer </font> sollen sich gefälligst darum kümmern , daß an demokratische Rechtsstaaten geliefert wird .

###




Evaluate with probabilities

In [63]:
# text = ' '.join(validation_df.loc[422, 'tokens'])
# text = 'Der Schneider begrüßt Klaus Schneider und den anderen Schneider.'
text = ' '.join(validation_df.loc[502, 'tokens'])

#text = 'Wir suchen zum nächstmöglichen Zeitpunkt einen Schreiner für unseren Familienbetrieb!'
#text = 'Rentner und Senioren müssen es sich noch leisten können.'
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]

probs = logits.numpy()
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'].flatten())
for i, t in enumerate(tokens):
    prob_o = probs[0,i,0]
    prob_prof = probs[0,i,1]
    prob_dif = prob_prof - prob_o
    print('{}\t{}\t{}\t'.format(predicted_token_class[i], prob_dif, t))

O	-9.15356731414795	[CLS]	
O	-10.083218574523926	Um	
O	-10.705368995666504	die	
O	-10.18628215789795	notwendige	
O	-7.721098899841309	Aufklärung	
O	-10.03056526184082	und	
O	-7.935290336608887	Beratung	
O	-10.74579906463623	sicherzustellen	
O	-10.911306381225586	,	
O	-10.669681549072266	sind	
O	-10.016093254089355	Regelungen	
O	-9.923088073730469	über	
O	-9.833395957946777	die	
O	-8.96151351928711	fach	
O	-9.270217895507812	##lichen	
O	-7.808472633361816	Kenntnisse	
O	-6.193926811218262	der	
B-PROF	1.0664318799972534	Verkäufer	
O	-5.115087509155273	im	
O	-3.9412169456481934	Einzelhandel	
O	-10.498884201049805	zu	
O	-10.574868202209473	treffen	
O	-10.373981475830078	.	
O	-7.675734519958496	[SEP]	
